In [3]:
import pandas as pd

In [4]:
df_train = pd.read_csv("data/hackathon_loreal_train_set.csv")
df_train

,Unnamed: 0,text,skincare,hair,make-up,other,index
0,0,Skeeterbytes wrote: Bassam Guy wrote: I don't ...,0.0,0.0,0.0,1.0,0
1,1,Refueling before the long week 🍵 - You’re mean...,0.0,0.0,0.0,1.0,1
2,2,Just in case no one has told you today YOU ARE...,0.0,0.0,1.0,0.0,2
3,3,Do not forget to get some sunshine safely if y...,0.0,0.0,0.0,1.0,3
4,4,Abstract Background Humans have dramatically c...,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...,...
10567,13209,☑ Day: 273 Outifit: Thor (Gold Foil) Back Blin...,0.0,0.0,0.0,1.0,13209
10568,13210,In order to study the various trends and patte...,0.0,0.0,0.0,1.0,13210
10569,13212,This brand has not yet registered with Influen...,0.0,0.0,0.0,1.0,13212
10570,13213,here is my regime get rid of any acne scars an...,1.0,0.0,0.0,0.0,13213


# watch data

In [5]:
df_train.columns

Index(['Unnamed: 0', 'text', 'skincare', 'hair', 'make-up', 'other', 'index'], dtype='object')

In [6]:
df = df_train[df_train['Unnamed: 0'] != df_train['index']]
df

,Unnamed: 0,text,skincare,hair,make-up,other,index


# delete one columns

In [7]:
df_train = df_train.drop(['Unnamed: 0'], axis = 1)
df_train

,text,skincare,hair,make-up,other,index
0,Skeeterbytes wrote: Bassam Guy wrote: I don't ...,0.0,0.0,0.0,1.0,0
1,Refueling before the long week 🍵 - You’re mean...,0.0,0.0,0.0,1.0,1
2,Just in case no one has told you today YOU ARE...,0.0,0.0,1.0,0.0,2
3,Do not forget to get some sunshine safely if y...,0.0,0.0,0.0,1.0,3
4,Abstract Background Humans have dramatically c...,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...
10567,☑ Day: 273 Outifit: Thor (Gold Foil) Back Blin...,0.0,0.0,0.0,1.0,13209
10568,In order to study the various trends and patte...,0.0,0.0,0.0,1.0,13210
10569,This brand has not yet registered with Influen...,0.0,0.0,0.0,1.0,13212
10570,here is my regime get rid of any acne scars an...,1.0,0.0,0.0,0.0,13213


# preprocessing one text

In [8]:
import pandas as pd

import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.metrics import ConfusionMatrix
from nltk.stem.snowball import SnowballStemmer

from nltk import word_tokenize, WordNetLemmatizer, PorterStemmer
from nltk import pos_tag
from nltk import ngrams
from nltk import sent_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
Onetext = df_train['text'].iloc[1]
Onetext

'Refueling before the long week 🍵 - You’re meant to Drink it, but I prefer it in my 🍵 From to , matcha is a natural powder of freshly ground leaves! ✅thanks to its numerous antioxidant benefits! ✅Including it in your beauty routine requires very little time and effort to see benefits ✅ The best part of it is that it’s already in your cupboard and doubles as a delicious drink? It really doesn’t get better than this! 🛒 Matcha tea purchased from my favorite online store faithfultonature'

In [10]:
import re

# Function to remove emoji.
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [11]:
emoji(Onetext)

'Refueling before the long week  - You’re meant to Drink it, but I prefer it in my  From to , matcha is a natural powder of freshly ground leaves! thanks to its numerous antioxidant benefits! Including it in your beauty routine requires very little time and effort to see benefits  The best part of it is that it’s already in your cupboard and doubles as a delicious drink? It really doesn’t get better than this!  Matcha tea purchased from my favorite online store faithfultonature'

In [30]:
OnetextLower = Onetext.lower()
OnetextLower

'skeeterbytes wrote: bassam guy wrote: i don\'t have anything longer than 60mm except my $99 "plastic [not so] fantastic" 40-150. i\'ve never really tried avian photography before but would like to get some decent shots (unlike these) when the opportunity arises. i used c-af+tr. in retrospect, since the turkey lawyer refused to fly despite my pleas. would s-af have done better? mf, peaking or magnify or both? i exposed for the lawyer, dressed in dark suits against a bright cloudy sky (some meteorologists say it is smoke remnants from ca in northern va). was +2 and +2.5 ev too much and encouraged the chroma? i\'m not great at judging distance but i would say i was about 35+ meters away. pretty intense purple fringing. lightroom has an effective one-click remover and i\'m sure other software has, too. that should not be hard to control. i know how to do that. i own the panasonic 7-14 4 never use c-af+tr myself, preferring standard c-af or s-af. in this instance, s-af is fine because your

In [31]:
tokenizeSingletext = word_tokenize(OnetextLower)
tokenizeSingletext

['skeeterbytes',
 'wrote',
 ':',
 'bassam',
 'guy',
 'wrote',
 ':',
 'i',
 'do',
 "n't",
 'have',
 'anything',
 'longer',
 'than',
 '60mm',
 'except',
 'my',
 '$',
 '99',
 '``',
 'plastic',
 '[',
 'not',
 'so',
 ']',
 'fantastic',
 "''",
 '40-150.',
 'i',
 "'ve",
 'never',
 'really',
 'tried',
 'avian',
 'photography',
 'before',
 'but',
 'would',
 'like',
 'to',
 'get',
 'some',
 'decent',
 'shots',
 '(',
 'unlike',
 'these',
 ')',
 'when',
 'the',
 'opportunity',
 'arises',
 '.',
 'i',
 'used',
 'c-af+tr',
 '.',
 'in',
 'retrospect',
 ',',
 'since',
 'the',
 'turkey',
 'lawyer',
 'refused',
 'to',
 'fly',
 'despite',
 'my',
 'pleas',
 '.',
 'would',
 's-af',
 'have',
 'done',
 'better',
 '?',
 'mf',
 ',',
 'peaking',
 'or',
 'magnify',
 'or',
 'both',
 '?',
 'i',
 'exposed',
 'for',
 'the',
 'lawyer',
 ',',
 'dressed',
 'in',
 'dark',
 'suits',
 'against',
 'a',
 'bright',
 'cloudy',
 'sky',
 '(',
 'some',
 'meteorologists',
 'say',
 'it',
 'is',
 'smoke',
 'remnants',
 'from',
 'ca'

In [32]:
lemmatizer = WordNetLemmatizer()
lemmatizeSingleText = list(map(lemmatizer.lemmatize, tokenizeSingletext))
lemmatizeSingleText

['skeeterbytes',
 'wrote',
 ':',
 'bassam',
 'guy',
 'wrote',
 ':',
 'i',
 'do',
 "n't",
 'have',
 'anything',
 'longer',
 'than',
 '60mm',
 'except',
 'my',
 '$',
 '99',
 '``',
 'plastic',
 '[',
 'not',
 'so',
 ']',
 'fantastic',
 "''",
 '40-150.',
 'i',
 "'ve",
 'never',
 'really',
 'tried',
 'avian',
 'photography',
 'before',
 'but',
 'would',
 'like',
 'to',
 'get',
 'some',
 'decent',
 'shot',
 '(',
 'unlike',
 'these',
 ')',
 'when',
 'the',
 'opportunity',
 'arises',
 '.',
 'i',
 'used',
 'c-af+tr',
 '.',
 'in',
 'retrospect',
 ',',
 'since',
 'the',
 'turkey',
 'lawyer',
 'refused',
 'to',
 'fly',
 'despite',
 'my',
 'plea',
 '.',
 'would',
 's-af',
 'have',
 'done',
 'better',
 '?',
 'mf',
 ',',
 'peaking',
 'or',
 'magnify',
 'or',
 'both',
 '?',
 'i',
 'exposed',
 'for',
 'the',
 'lawyer',
 ',',
 'dressed',
 'in',
 'dark',
 'suit',
 'against',
 'a',
 'bright',
 'cloudy',
 'sky',
 '(',
 'some',
 'meteorologist',
 'say',
 'it',
 'is',
 'smoke',
 'remnant',
 'from',
 'ca',
 'i

In [33]:
stop_punctuation = [':', '(', ')', '/', '|', ',', ']', ';',
                    '.', '*', '#', '"', '&', '~', '``',
                    '-', '_', '\\', '@','?','!','\'', '[']
TextCleanPonctuation = [word for word in lemmatizeSingleText if word not in stop_punctuation]
TextCleanPonctuation

['skeeterbytes',
 'wrote',
 'bassam',
 'guy',
 'wrote',
 'i',
 'do',
 "n't",
 'have',
 'anything',
 'longer',
 'than',
 '60mm',
 'except',
 'my',
 '$',
 '99',
 'plastic',
 'not',
 'so',
 'fantastic',
 "''",
 '40-150.',
 'i',
 "'ve",
 'never',
 'really',
 'tried',
 'avian',
 'photography',
 'before',
 'but',
 'would',
 'like',
 'to',
 'get',
 'some',
 'decent',
 'shot',
 'unlike',
 'these',
 'when',
 'the',
 'opportunity',
 'arises',
 'i',
 'used',
 'c-af+tr',
 'in',
 'retrospect',
 'since',
 'the',
 'turkey',
 'lawyer',
 'refused',
 'to',
 'fly',
 'despite',
 'my',
 'plea',
 'would',
 's-af',
 'have',
 'done',
 'better',
 'mf',
 'peaking',
 'or',
 'magnify',
 'or',
 'both',
 'i',
 'exposed',
 'for',
 'the',
 'lawyer',
 'dressed',
 'in',
 'dark',
 'suit',
 'against',
 'a',
 'bright',
 'cloudy',
 'sky',
 'some',
 'meteorologist',
 'say',
 'it',
 'is',
 'smoke',
 'remnant',
 'from',
 'ca',
 'in',
 'northern',
 'va',
 'wa',
 '+2',
 'and',
 '+2.5',
 'ev',
 'too',
 'much',
 'and',
 'encourag

In [34]:
stoplist = stopwords.words('english')
TextCleanPonctuationWord = [word for word in TextCleanPonctuation if word not in stoplist]
TextCleanPonctuationWord

['skeeterbytes',
 'wrote',
 'bassam',
 'guy',
 'wrote',
 "n't",
 'anything',
 'longer',
 '60mm',
 'except',
 '$',
 '99',
 'plastic',
 'fantastic',
 "''",
 '40-150.',
 "'ve",
 'never',
 'really',
 'tried',
 'avian',
 'photography',
 'would',
 'like',
 'get',
 'decent',
 'shot',
 'unlike',
 'opportunity',
 'arises',
 'used',
 'c-af+tr',
 'retrospect',
 'since',
 'turkey',
 'lawyer',
 'refused',
 'fly',
 'despite',
 'plea',
 'would',
 's-af',
 'done',
 'better',
 'mf',
 'peaking',
 'magnify',
 'exposed',
 'lawyer',
 'dressed',
 'dark',
 'suit',
 'bright',
 'cloudy',
 'sky',
 'meteorologist',
 'say',
 'smoke',
 'remnant',
 'ca',
 'northern',
 'va',
 'wa',
 '+2',
 '+2.5',
 'ev',
 'much',
 'encouraged',
 'chroma',
 "'m",
 'great',
 'judging',
 'distance',
 'would',
 'say',
 'wa',
 '35+',
 'meter',
 'away',
 'pretty',
 'intense',
 'purple',
 'fringing',
 'lightroom',
 'ha',
 'effective',
 'one-click',
 'remover',
 "'m",
 'sure',
 'software',
 'ha',
 'hard',
 'control',
 'know',
 'panasonic',


# functions

In [35]:
def preprocessing(singleText):
    lemmatizer = WordNetLemmatizer()
    stop_punctuation = [':', '(', ')', '/', '|', ',', ']', ';',
                    '.', '*', '#', '"', '&', '~', '``',
                    '-', '_', '\\', '@','?','!','\'', '[']
    stoplist = stopwords.words('english')

    LowerSingleText = singleText.lower()
    tokenizeSingletext = word_tokenize(LowerSingleText)
    lemmatizeSingleText = list(map(lemmatizer.lemmatize, tokenizeSingletext))
    
    TextCleanPonctuation = [word for word in lemmatizeSingleText if word not in stop_punctuation and word not in stoplist]

    return TextCleanPonctuation

def clean_text(text):
    text_preprocessed = preprocessing(text)
    
    clean_mail = [word for word in text_preprocessed if word not in stop_punctuation and not word in stoplist]
    return ' '.join(clean_mail)

In [36]:
result = clean_text(df_train["text"].iloc[0])
result

"skeeterbytes wrote bassam guy wrote n't anything longer 60mm except $ 99 plastic fantastic '' 40-150. 've never really tried avian photography would like get decent shot unlike opportunity arises used c-af+tr retrospect since turkey lawyer refused fly despite plea would s-af done better mf peaking magnify exposed lawyer dressed dark suit bright cloudy sky meteorologist say smoke remnant ca northern va wa +2 +2.5 ev much encouraged chroma 'm great judging distance would say wa 35+ meter away pretty intense purple fringing lightroom ha effective one-click remover 'm sure software ha hard control know panasonic 7-14 4 never use c-af+tr preferring standard c-af s-af instance s-af fine attorney n't going anywhere pay bill get s-af+magnify allow tweak focus plus want stop lens bit sharpen except 7-14 nothing else f2.8 5.6 seems stopped 'll watch tried magnify wa shaky long fl 'll try lower scale factor also raise shutter speed freeze flapping 're iso200 's good lens within realm 40-150 pro 

# On applique cela pour toute la base de donnée

In [37]:
df_train["clean_content"] = df_train.text.apply(clean_text)
df_train["clean_content"]

0        skeeterbytes wrote bassam guy wrote n't anythi...
1        refueling long week 🍵 ’ meant drink prefer 🍵 m...
2        case one ha told today strong beautiful love 🌹...
3        forget get sunshine safely ☀️ head link bio se...
4        abstract background human dramatically changed...
                               ...                        
10567    ☑ day 273 outifit thor gold foil back bling et...
10568    order study various trend pattern prevailing c...
10569    brand ha yet registered influenster work brand...
10570    regime get rid acne scar get way smoother heal...
10571    's true used work caddie trump national nj don...
Name: clean_content, Length: 10572, dtype: object

# vectorization

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

## separate

In [41]:
X = df_train['clean_content']
Y = df_train.drop(['clean_content', 'index'], axis = 1)
X

0        skeeterbytes wrote bassam guy wrote n't anythi...
1        refueling long week 🍵 ’ meant drink prefer 🍵 m...
2        case one ha told today strong beautiful love 🌹...
3        forget get sunshine safely ☀️ head link bio se...
4        abstract background human dramatically changed...
                               ...                        
10567    ☑ day 273 outifit thor gold foil back bling et...
10568    order study various trend pattern prevailing c...
10569    brand ha yet registered influenster work brand...
10570    regime get rid acne scar get way smoother heal...
10571    's true used work caddie trump national nj don...
Name: clean_content, Length: 10572, dtype: object

In [42]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [43]:
pipe = make_pipeline(CountVectorizer(), TfidfTransformer())
pipe.fit(x_train)
feat_train = pipe.transform(x_train)
feat_train

<8457x71330 sparse matrix of type '<class 'numpy.float64'>'
	with 704785 stored elements in Compressed Sparse Row format>

In [44]:
feat_train

<8457x71330 sparse matrix of type '<class 'numpy.float64'>'
	with 704785 stored elements in Compressed Sparse Row format>

# Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50)
clf.fit(feat_train, y_train)

In [ ]:
feat_test = pipe.transform(x_test)
clf.score(feat_test, y_test)

# visualisation result

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
score = clf.predict_proba(feat_test)
score
#fpr, tpr, th = roc_curve(y_list_test, score[:, 1])

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(4,4))
ax.plot([0, 1], [0, 1], 'k--')
aucf = auc(fpr, tpr)
ax.plot(fpr, tpr, label='auc=%1.5f' % aucf)
ax.set_title('Courbe ROC - classifieur de sentiments')
ax.legend();